# Imports

In [7]:
import pandas as pd
import os

# Variables

In [20]:
footballdata_wIndexes_path = os.path.abspath(os.getcwd())+"\\Java\\football-data_wIndexes.csv"
idx_mappings_dir = os.path.abspath(os.getcwd())+"\\Data\\footballAPI_idx_mappings\\englishpremierleague.csv"

# Load Football-Data File with Indexes

In [21]:
footballdata_wIndexes = pd.read_csv(footballdata_wIndexes_path, delimiter=";")
footballdata_wIndexes

,Date,HomeTeamid,HomeTeamName,AwayTeamId,AwayTeamName,HomeGoals,AwayGoals,Unnamed: 7
0,14/08/2010,598,West Ham,0,3,0,H,NaN
1,14/08/2010,8560,Everton,0,1,0,H,NaN
2,14/08/2010,535,Fulham,0,0,0,D,NaN
3,14/08/2010,597,West Brom,0,6,0,H,NaN
4,14/08/2010,12596,Birmingham,0,2,2,D,NaN
...,...,...,...,...,...,...,...,...
4175,23/05/2021,550,Liverpool,525,Crystal Palace,2,0,NaN
4176,23/05/2021,555,Man City,8560,Everton,5,0,NaN
4177,23/05/2021,0,Sheffield United,512,Burnley,1,0,NaN
4178,23/05/2021,598,West Ham,585,Southampton,3,0,NaN


## Store Index Mapping

In [14]:
idxs = footballdata_wIndexes[['HomeTeamName','HomeTeamid']].drop_duplicates().set_index('HomeTeamName')\
    .sort_values('HomeTeamName')
idxs

,HomeTeamid
HomeTeamName,
Arsenal,494
Aston Villa,495
Brentford,10503
Brighton,508
Burnley,512
Chelsea,519
Crystal Palace,525
Everton,8560
Leeds,546


In [19]:
idxs.to_csv(idx_mappings_dir)

# Call API to get match Stats

In [ ]:
def 